In [1]:
from sklearn.datasets import load_boston
import numpy as np
import matplotlib.pyplot as plt
import warnings
from sklearn.model_selection import KFold
from sklearn.preprocessing import PolynomialFeatures

## Avoid printing out warnings

with warnings.catch_warnings():
     warnings.filterwarnings("ignore")
     X, y = load_boston(return_X_y=True)

In [2]:
X.shape

(506, 13)

506 examples and 13 features

# Implementing Normal Equation Solution

In [3]:
kf = KFold(n_splits = 10)
train_mse = []
test_mse = []

for train_index, test_index in kf.split(X):
    # Splitting the dataset into train set and test set
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    m_train, m_test = X_train.shape[0], X_test.shape[0]

    # Appending a column of ones to X_train and X_test to account for the bias term 
    X_train = np.append(X_train, np.ones((m_train,1)), axis = 1)
    X_test = np.append(X_test, np.ones((m_test, 1)), axis=1)

    # Reshaping y_train and y_test
    y_train = y_train.reshape(m_train,1)
    y_test = y_test.reshape(m_test,1)

    # Computing Theta using the Normal Equation (Closed-Form Solution)
    theta = np.dot(np.linalg.inv(np.dot(X_train.T, X_train)), np.dot(X_train.T, y_train))

    # Evaluating the model on the train set
    y_train_hat = X_train.dot(theta)
    mse_train = (1/m_train) * np.sum((y_train_hat - y_train)**2)

    # Evaluating the model on the test set
    y_test_hat = X_test.dot(theta)
    mse_test = (1/m_test) * np.sum((y_test_hat - y_test)**2)

    # Storing the results
    train_mse.append(mse_train)
    test_mse.append(mse_test)

avg_train_mse = np.mean(train_mse)
avg_test_mse = np.mean(test_mse)

print("Average Training MSE:", avg_train_mse)
print("Average Test MSE:", avg_test_mse)


Average Training MSE: 21.39939241710581
Average Test MSE: 34.705255944527316


# Implementing a Ridge Regression Model

In [4]:
kf = KFold(n_splits = 10)
lambda_vals = np.logspace(1,7,num=13)
best_lambda = None
lowest_test_j = float('inf')

for lam_val in lambda_vals:
    j_train = []
    j_test = []
    
    for train_index, test_index in kf.split(X):
        # Splitting the dataset into train set and test set
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        m_train, m_test = X_train.shape[0], X_test.shape[0]
    
        # Appending a column of ones to X_train and X_test to account for the bias term 
        X_train = np.append(X_train, np.ones((m_train,1)), axis = 1)
        X_test = np.append(X_test, np.ones((m_test, 1)), axis=1)
    
        # Reshaping y_train and y_test
        y_train = y_train.reshape(m_train,1)
        y_test = y_test.reshape(m_test,1)

        # Creating identity matrix for regularization
        I = np.eye(X_train.shape[1])
        I[-1, -1] = 0 # This is done as the bias term should not be regulaized
    
        # Computing Theta using the Normal Equation (Closed-Form Solution)
        theta = np.dot(np.linalg.inv((np.dot(X_train.T, X_train)) + lam_val*I), np.dot(X_train.T, y_train))
    
        # Evaluating the model on the train set
        y_train_hat = X_train.dot(theta)
        mse_train = (1/m_train) * np.sum((y_train_hat - y_train)**2)
        j_train.append(mse_train + lam_val*0.5*np.sum(theta**2))
    
        # Evaluating the model on the test set
        y_test_hat = X_test.dot(theta)
        mse_test = (1/m_test) * np.sum((y_test_hat - y_test)**2)
        j_test .append(mse_test)

    avg_j_train = np.mean(j_train)
    avg_j_test = np.mean(j_test)

    if avg_j_test < lowest_test_j:
        lowest_test_j = avg_j_test
        best_lambda = lam_val

print("The best λ value is: ", best_lambda)
print("The corresponding test error is: ", lowest_test_j)

The best λ value is:  100.0
The corresponding test error is:  29.61522009733349


# Estimaing the Performace of Model with Optimal λ

In [5]:
kf = KFold(n_splits = 10)
train_mse = []
test_mse = []

for train_index, test_index in kf.split(X):
    # Splitting the dataset into train set and test set
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    m_train, m_test = X_train.shape[0], X_test.shape[0]

    # Appending a column of ones to X_train and X_test to account for the bias term 
    X_train = np.append(X_train, np.ones((m_train,1)), axis = 1)
    X_test = np.append(X_test, np.ones((m_test, 1)), axis=1)

    # Reshaping y_train and y_test
    y_train = y_train.reshape(m_train,1)
    y_test = y_test.reshape(m_test,1)

    # Creating identity matrix for regularization
    I = np.eye(X_train.shape[1])
    I[-1, -1] = 0 # This is done as the bias term should not be regulaized

    # Computing Theta using the Normal Equation (Closed-Form Solution)
    theta = np.dot(np.linalg.inv((np.dot(X_train.T, X_train)) + best_lambda*I), np.dot(X_train.T, y_train))

    # Evaluating the model on the train set
    y_train_hat = X_train.dot(theta)
    mse_train = (1/m_train) * np.sum((y_train_hat - y_train)**2)
    train_mse.append(mse_train)
    
    # Evaluating the model on the test set
    y_test_hat = X_test.dot(theta)
    mse_test = (1/m_test) * np.sum((y_test_hat - y_test)**2)
    test_mse.append(mse_test)

avg_train_mse = np.mean(train_mse)
avg_test_mse = np.mean(test_mse)

print("Average Training Score:", avg_train_mse)
print("Average Test Score:", avg_test_mse)


Average Training Score: 23.609927954689702
Average Test Score: 29.61522009733349


# Implementing a Polynomial Transformation of Degree 2

In [6]:
kf = KFold(n_splits = 10)
train_mse = []
test_mse = []

for train_index, test_index in kf.split(X):
    # Splitting the dataset into train set and test set
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    m_train, m_test = X_train.shape[0], X_test.shape[0]

    # Polynomial Transformation
    poly = PolynomialFeatures(degree = 2, include_bias = False)
    X_train_poly = poly.fit_transform(X_train)
    X_test_poly = poly.transform(X_test)

    # Appending a column of ones to X_train and X_test to account for the bias term 
    X_train_poly = np.append(X_train_poly, np.ones((m_train,1)), axis = 1)
    X_test_poly = np.append(X_test_poly, np.ones((m_test, 1)), axis=1)

    # Reshaping y_train and y_test
    y_train = y_train.reshape(m_train,1)
    y_test = y_test.reshape(m_test,1)

    # Creating identity matrix for regularization
    I = np.eye(X_train_poly.shape[1])
    I[-1, -1] = 0 # This is done as the bias term should not be regulaized

    # Computing Theta using the Normal Equation (Closed-Form Solution)
    theta = np.dot(np.linalg.inv((np.dot(X_train_poly.T, X_train_poly))+ best_lambda*I), np.dot(X_train_poly.T, y_train))

    # Evaluating the model on the train set
    y_train_hat = X_train_poly.dot(theta)
    mse_train = (1/m_train) * np.sum((y_train_hat - y_train)**2)
    train_mse.append(mse_train)
    
    # Evaluating the model on the test set
    y_test_hat = X_test_poly.dot(theta)
    mse_test = (1/m_test) * np.sum((y_test_hat - y_test)**2)
    test_mse.append(mse_test)

avg_train_mse = np.mean(train_mse)
avg_test_mse = np.mean(test_mse)

print("Average Training Score:", avg_train_mse)
print("Average Test Score:", avg_test_mse)


Average Training Score: 7.481646104598366
Average Test Score: 65.55557808784023


# Implementing Multivariate Linear Regression using the Gradient Descent method

In [12]:
# Scale the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

kf = KFold(n_splits=10)
train_mse = []
test_mse = []

for train_index, test_index in kf.split(X):
    # Splitting the dataset into train set and test set
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    m_train, m_test = X_train.shape[0], X_test.shape[0]

    # Appending a column of ones to X_train and X_test to account for the bias term
    X_train = np.append(X_train, np.ones((m_train, 1)), axis=1)
    X_test = np.append(X_test, np.ones((m_test, 1)), axis=1)

    # Reshaping y_train and y_test
    y_train = y_train.reshape(m_train, 1)
    y_test = y_test.reshape(m_test, 1)

    # Hyperparameters
    eta = 0.01  
    n_iterations = 10_000
    tolerance = 1e-8

    # Random Initialization
    theta = np.random.randn(X_train.shape[1], 1)

    # Gradient Descent
    for i in range(n_iterations):
        gradients = (2/m_train) * (X_train.T).dot(X_train.dot(theta) - y_train)
        theta = theta - eta * gradients

        if np.linalg.norm(gradients) < tolerance:
            break

    # Evaluating the model on the train set
    y_train_hat = X_train.dot(theta)
    mse_train = (1/m_train) * np.sum((y_train_hat - y_train)**2)
    train_mse.append(mse_train)
    
    # Evaluating the model on the test set
    y_test_hat = X_test.dot(theta)
    mse_test = (1/m_test) * np.sum((y_test_hat - y_test)**2)
    test_mse.append(mse_test)

# Calculating average MSE
avg_train_mse = np.mean(train_mse)
avg_test_mse = np.mean(test_mse)

print("Average Training MSE:", avg_train_mse)
print("Average Test MSE:", avg_test_mse)

Average Training MSE: 21.3993924174482
Average Test MSE: 34.705267719600265
